In [151]:
import numpy as np
import torch

def wrap(x):
    return np.remainder(x + np.pi, np.pi * 2) - np.pi

def place_fourth_atom(a, b, c, bond_angle, torsion, bond_length):
    # Place atom D with respect to atom C at origin.
    d = np.array([
        bond_length * np.cos(np.pi - bond_angle),
        bond_length * np.cos(torsion) * np.sin(bond_angle),
        bond_length * np.sin(torsion) * np.sin(bond_angle)
    ]).T

    # Transform atom D to the correct frame.
    bc = c - b
    bc /= np.linalg.norm(bc) # Unit vector from B to C.

    n = np.cross(b - a, bc)
    n /= np.linalg.norm(n) # Normal vector of the plane defined by a, b, c.

    M = np.array([bc, np.cross(n, bc), n]).T
    return M @ d + c

def angles2coord(angles, n_ca=1.46, ca_c=1.54, c_n=1.33):
    """Given L x 6 angle matrix,
    reconstruct the Cartesian coordinates of atoms.
    Returns L x 3 coordinate matrix.

    Implements NeRF (Natural Extension Reference Frame) algorithm.
    """
    if isinstance(angles, torch.Tensor):
        phi, psi, omega, theta1, theta2, theta3 = angles.T.numpy()
    else:
        phi, psi, omega, theta1, theta2, theta3 = angles.T
    
    torsions = np.stack([psi[:-1], omega[:-1], phi[1:]], axis=-1).flatten()
    bond_angles = np.stack([theta2[:-1], theta3[:-1], theta1[1:]], axis=-1).flatten()
    
    #
    # Place the first three atoms.
    #
    # The first atom (N) is placed at origin.
    a = np.zeros(3)
    # The second atom (Ca) is placed on the x-axis.
    b = np.array([1, 0, 0]) * n_ca
    # The third atom (C) is placed on the xy-plane with bond angle theta1[0]
    c = np.array([ np.cos(np.pi - theta1[0]), np.sin(np.pi - theta1[0]), 0 ]) * ca_c + b

    # Iteratively place the fourth atom based on the last three atoms.

    coords = [a, b, c]
    # cycle through [n, ca, c, n, ca, c, ...]
    for i, bond_length in enumerate([c_n, n_ca, ca_c] * (len(angles) - 1)):
        torsion, bond_angle = torsions[i], bond_angles[i]
        d = place_fourth_atom(a, b, c, bond_angle, torsion, bond_length)
        coords.append(d)

        a, b, c = b, c, d
    
    return np.array(coords)

def beta(t, T=1000, s=8e-3):
    f_t = np.cos( (t/T + s) / (1 + s) * np.pi * 0.5 ) ** 2
    f_t_minus1 = np.cos( ((t-1)/T + s) / (1 + s) * np.pi * 0.5 ) ** 2
    
    b = np.clip(1 - f_t / f_t_minus1, a_min=1e-5, a_max=1 - 1e-5)
    return b

In [134]:
from biotite.structure.io.pdb import PDBFile
import biotite.structure as struc
from tqdm import tqdm

In [162]:
np.repeat(range(10), 3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7,
       7, 7, 8, 8, 8, 9, 9, 9])

In [163]:
import itertools

for coords in [angles2coord(angles)]:
    structure = struc.AtomArray(len(coords))
    structure.coord = coords
    structure.atom_name = ['N', 'CA', 'C'] * (len(coords) // 3)
    structure.res_id = np.repeat( range(1, len(coords) // 3 + 1), 3 )
            
    pdb = PDBFile()
    pdb.set_structure(structure)
    pdb.write('test.pdb')
    
    break

In [165]:
angles = np.load('../data/npy/1a00B00.npy')

pdb = PDBFile.read('../data/dompdb/1a00B00.pdb')
structure = pdb.get_structure(model=1)
backbone = structure[struc.filter_backbone(structure)]

angles_list = [ angles ]
coords_list = [ angles2coord(angles) ]
for t in tqdm(range(1, 1001)):
    eps = np.random.normal(scale=np.sqrt(beta(t)), size=angles.shape)
    angles = angles + eps
    angles_list.append( angles )
    coords_list.append( angles2coord(angles) )

100%|██████████| 1000/1000 [00:51<00:00, 19.60it/s]


In [167]:
for i, coords in tqdm(enumerate(coords_list)):
    num_residues = len(coords) // 3
    
    structure = struc.AtomArray(len(coords))
    structure.coord = coords
    structure.atom_name = ['N', 'CA', 'C'] * (num_residues)
    structure.res_name = ['GLY'] * (len(coords))
    structure.res_id = np.repeat( range(1, num_residues + 1), 3 )
    
    pdb = PDBFile()
    pdb.set_structure(structure)
    pdb.write(f'noising_demo/model_{i}.pdb')

1001it [00:12, 80.66it/s]


In [92]:
struc.distance(backbone[1], backbone[2])

1.5280697

In [95]:
struc.distance(coords_list[0][1], coords_list[0][2])

1.5400001

In [106]:
angles2coord(angles)

array([[  0.        ,   0.        ,   0.        ],
       [  1.46      ,   0.        ,   0.        ],
       [  1.95978841,   0.        ,   1.45664393],
       ...,
       [ -7.46618621,  -6.33037823, 189.34789747],
       [ -6.48831196,  -6.82150823, 190.431481  ],
       [ -6.77035415,  -7.7042811 , 191.38545388]])

In [99]:
import itertools

for coords in [angles2coord(angles)]:
    structure = struc.AtomArray(len(coords))
    structure.coord = coords
    structure.atom_name = ['N', 'CA', 'C'] * (len(coords) // 3)
            
    pdb = PDBFile()
    pdb.set_structure(structure)
    pdb.write('test.pdb')
    
    break

In [ ]:
struc

In [70]:
structure.set_annotation()

array([nan, nan, nan], dtype=float32)

In [57]:
structure

array([Atom(np.array([0., 0., 0.], dtype=float32), chain_id="", res_id=0, ins_code="", res_name="", hetero=False, atom_name="", element=""),
       Atom(np.array([1.46, 0.  , 0.  ], dtype=float32), chain_id="", res_id=0, ins_code="", res_name="", hetero=False, atom_name="", element=""),
       Atom(np.array([2.4159732, 0.       , 1.2073588], dtype=float32), chain_id="", res_id=0, ins_code="", res_name="", hetero=False, atom_name="", element=""),
       Atom(np.array([ 3.1496563 , -0.38536483,  2.4093583 ], dtype=float32), chain_id="", res_id=0, ins_code="", res_name="", hetero=False, atom_name="", element=""),
       Atom(np.array([1.9895512 , 0.26409912, 1.6322145 ], dtype=float32), chain_id="", res_id=0, ins_code="", res_name="", hetero=False, atom_name="", element=""),
       Atom(np.array([2.4412527, 1.3248187, 0.9690859], dtype=float32), chain_id="", res_id=0, ins_code="", res_name="", hetero=False, atom_name="", element=""),
       Atom(np.array([ 2.1523347 , -0.00824691,  1.4897

In [12]:
eps

array([[-2.61644690e-01, -1.94743684e+00, -3.88029499e-02,
        -9.00576467e-01, -1.99345270e+00, -1.39806284e+00],
       [ 3.90051074e-01, -3.04791030e-01,  1.94887889e+00,
         5.55575973e-01, -2.08452838e-01,  3.07781015e-01],
       [ 2.29117829e-01,  1.25494543e+00,  9.02436788e-01,
        -6.08868986e-01,  1.56102670e+00, -1.06102125e+00],
       [ 1.56628724e+00, -2.03578182e+00, -1.19140309e+00,
         5.55938122e-01, -1.53734465e+00, -7.69076786e-01],
       [-1.20804396e-01, -3.86620033e-01, -1.17056689e+00,
        -6.50596316e-01,  4.00160010e-01, -1.53707915e+00],
       [ 1.10687221e+00,  9.94686743e-01,  2.19308668e-01,
         5.96399513e-01, -1.93509417e+00,  7.82962580e-01],
       [-5.72774630e-01, -7.42206418e-01, -3.04665522e-01,
        -1.91693707e+00,  5.75992041e-01, -9.61933329e-03],
       [ 2.59921636e+00,  3.92634189e-01, -1.57079760e-01,
         1.53554433e-01,  9.16871846e-01, -6.25157991e-01],
       [ 8.13981109e-02, -5.17342074e-01,  7.042

In [7]:
backbone

array([
	Atom(np.array([68.023, 46.313, -6.831], dtype=float32), chain_id="B", res_id=1, ins_code="", res_name="MET", hetero=False, atom_name="N", element="N"),
	Atom(np.array([68.102, 45.947, -8.262], dtype=float32), chain_id="B", res_id=1, ins_code="", res_name="MET", hetero=False, atom_name="CA", element="C"),
	Atom(np.array([66.983, 46.661, -9.019], dtype=float32), chain_id="B", res_id=1, ins_code="", res_name="MET", hetero=False, atom_name="C", element="C"),
	Atom(np.array([67.33 , 47.886, -9.382], dtype=float32), chain_id="B", res_id=2, ins_code="", res_name="HIS", hetero=False, atom_name="N", element="N"),
	Atom(np.array([ 66.381,  48.684, -10.165], dtype=float32), chain_id="B", res_id=2, ins_code="", res_name="HIS", hetero=False, atom_name="CA", element="C"),
	Atom(np.array([ 67.146,  49.102, -11.428], dtype=float32), chain_id="B", res_id=2, ins_code="", res_name="HIS", hetero=False, atom_name="C", element="C"),
	Atom(np.array([ 66.619,  48.602, -12.527], dtype=float32), chain_